In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..\\CommandFiles'))
	print(os.getcwd())
except:
	pass


In [15]:
import pandas as pd
import datetime
from os import listdir
from os.path import isfile, join
import glob
import re

In [16]:
file = open('./../OrginalData/weather.txt')
data = file.read()
file.close()

data = re.sub('[BDS]?[\t ]+(OI|OS|SI|I+|S+)[\t ]*',',',data)
data = re.sub('[\t ]+',',',data)
data = re.sub('\s*,[\t ]*\n','\n',data)
data = re.sub('(\w+)(\d\d\d\d)(\d\d)(\w\w\w\w)','\g<1>,\g<2>,\g<3>,\g<4>',data)
data = data.replace('-9999','')

#Create headers
header = 'id,year,month,element'
for i in range(1,32):
    header += ',D'+str(i)
    
#Connect headers with data
data = header+'\n'+data

#Save *.txt file as *.csv 
file = open('./../AnalysisData/weather_raw.csv','w')
file.write(data)
file.close()

In [17]:
df = pd.read_csv('./../AnalysisData/weather_raw.csv')
df = pd.melt(df, id_vars=['id', 'year', 'month', 'element'],
             value_vars=list(df.columns)[4:],
             var_name='day', value_name='value')
df['day'] = df['day'].str[1:].astype('int')
df['day'] = df[['day']].apply(
    lambda row: '{:02d}'.format(*row),
    axis=1)
df = df.loc[df['value'] != '---', ['id', 'year', 'month','day', 'element', 'value']]
df = df.set_index(['id', 'year', 'month', 'day', 'element'])
df = df.unstack()
df.columns = list(df.columns.get_level_values('element'))
df = df.reset_index()
df.to_csv('./../AnalysisData/weather_tidy.csv', index=False)

Rozwiązanie zadania
===================

 Podczas zadania nalezało odpowiedzieć na pytania:`Która stacja zarejestrował największą różnicę temperatur w ciągu miesiąca? Określ które ze stacji zarejestrowały temperatury najbliższe do średniej wartości temperatury maksymalnej i minimalnej z poszczególnych miesięcy.`

 Rozpocznijmy analizę danych

In [18]:
weather = pd.read_csv('./../AnalysisData/weather_tidy.csv')
weather.head()

,id,year,month,day,PRCP,TMAX,TMIN
0,MX000017004,1955,4,1,0.0,310.0,150.0
1,MX000017004,1955,4,2,0.0,310.0,150.0
2,MX000017004,1955,4,3,0.0,310.0,160.0
3,MX000017004,1955,4,4,0.0,320.0,150.0
4,MX000017004,1955,4,5,0.0,330.0,160.0


In [19]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18197 entries, 0 to 18196
Data columns (total 7 columns):
id       18197 non-null object
year     18197 non-null int64
month    18197 non-null int64
day      18197 non-null int64
PRCP     15440 non-null float64
TMAX     11274 non-null float64
TMIN     11995 non-null float64
dtypes: float64(3), int64(3), object(1)
memory usage: 995.3+ KB


In [20]:
weather.describe(include='all')

,id,year,month,day,PRCP,TMAX,TMIN
count,18197,18197.000000,18197.000000,18197.000000,15440.000000,11274.000000,11995.000000
unique,1,NaN,NaN,NaN,NaN,NaN,NaN
top,MX000017004,NaN,NaN,NaN,NaN,NaN,NaN
freq,18197,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,1983.035775,6.485520,16.000000,32.800065,271.502217,145.283535
std,NaN,17.073927,3.449072,8.944518,90.865970,31.225344,23.444385
min,NaN,1955.000000,1.000000,1.000000,0.000000,110.000000,0.000000
25%,NaN,1968.000000,3.000000,8.000000,0.000000,250.000000,130.000000
50%,NaN,1980.000000,6.000000,16.000000,0.000000,270.000000,150.000000
75%,NaN,1999.000000,9.000000,24.000000,10.000000,290.000000,160.000000


Odpowiedz
==========

 Zauważmy, że istnieje tylko jedno id, czyli jest tylko jedna stacja. Wynika z tego, że odpowiedziami na podane pytania jest ta jedna stacja.